In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia, tables_path
from bulletin.systems.sivep import Sivep
from glob import glob

In [ ]:
def macro_sort(column):
    """Sort function"""
    macros = ['Leste', 'Oeste', 'Noroeste', 'Norte', 'Fora']
    correspondence = {macro: order for order, macro in enumerate(macros)}
    return column.map(correspondence)


In [ ]:
tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])

In [ ]:
sivep = Sivep()
sivep.load('sraghospitalizado')
# sivep.read_all_database_files()
# sivep.save(replace=True)
sivep.df

In [ ]:
sivep.normalize()
sivep.dtypes

In [ ]:
sivep.df['DT_INTERNA'] = pd.to_datetime(sivep.df['DT_INTERNA'], errors='coerce', format='%Y-%m-%d')

In [ ]:
sivep.df['FATOR_RISC'] = sivep.df['FATOR_RISC'].astype(int)

In [ ]:
sivep.df['VACINA_COV'] = sivep.df['VACINA_COV'].fillna(2).astype(int)


In [ ]:
sivep.df['TP_IDADE'] = sivep.df['TP_IDADE'].astype(int)
sivep.df['NU_IDADE_N'] = sivep.df['NU_IDADE_N'].astype(int)

In [ ]:
sivep.df['SUPORT_VEN'] = sivep.df['SUPORT_VEN'].fillna(9).astype(int)

In [ ]:
relacao01 = sivep.df.loc[
    (sivep.df['DT_INTERNA'].dt.year == 2021) & 
    (sivep.df['SG_UF_INTE']=='PR') & 
    (sivep.df['CLASSI_FIN']==5) & 
    (sivep.df['FATOR_RISC']==2) & 
    (sivep.df['TP_IDADE']==3) & 
    (sivep.df['NU_IDADE_N']>=18) & 
    (sivep.df['NU_IDADE_N']<=43) & 
    (sivep.df['HOSPITAL']==1) & 
    (sivep.df['UTI']==1)  & 
    (sivep.df['VACINA_COV']==2) & 
    (sivep.df['EVOLUCAO']==2)
]
print(len(relacao01))

In [ ]:
relacao02 = sivep.df.loc[
    (sivep.df['DT_INTERNA'].dt.year == 2021) & 
    (sivep.df['SG_UF_INTE']=='PR') & 
    (sivep.df['CLASSI_FIN']==5) & 
    (sivep.df['FATOR_RISC']==2) & 
    (sivep.df['TP_IDADE']==3) & 
    (sivep.df['NU_IDADE_N']>=75) & 
    (sivep.df['HOSPITAL']==1) & 
    (sivep.df['UTI']==2)  & 
    (sivep.df['SUPORT_VEN'].isin([3,2])) & 
    (sivep.df['VACINA_COV']==2) & 
    (sivep.df['EVOLUCAO']==1)
]
print(len(relacao02))

In [ ]:
relacao01.to_pickle('sivep_2021_g1.pkl')

In [ ]:
relacao02.to_pickle('sivep_2021_g2.pkl')

In [ ]:
raise
DT_INTERNA = pd.to_datetime(sivep.df['DT_INTERNA'], errors='coerce').copy()
DT_INTERNA

In [ ]:
sivep.to_notifica()
sivep.df

In [ ]:
sivep.df = sivep.df.join(tables['municipios'].set_index('ibge'), on='ibge_residencia')

In [ ]:
sivep.df['data_entrada'] = DT_INTERNA

In [ ]:
sivep_confirmados = sivep.df.loc[sivep.df['classificacao_final']==2]

In [ ]:
sivep.df['ano_internamento'] = sivep.df.loc[sivep.df['data_entrada'].notna(),'data_entrada'].apply(lambda x: x.year)
sivep.df['mes_internamento'] = sivep.df.loc[sivep.df['data_entrada'].notna(),'data_entrada'].apply(lambda x: x.month)
sivep.df['se_internamento'] = sivep.df.loc[sivep.df['data_entrada'].notna(),'data_entrada'].apply(lambda x: Week.fromdate(x).week)
sivep.df['dia_internamento'] = sivep.df.loc[sivep.df['data_entrada'].notna(),'data_entrada'].apply(lambda x: x.day)

In [ ]:
localizacao = ['macro','rs','regional','ibge_residencia','municipio']
periodo = ['ano_internamento','mes_internamento','se_internamento','dia_internamento']

In [ ]:
sivep_confirmados

In [ ]:
sivep_mun_periodo = sivep_confirmados.groupby(localizacao+periodo)[['sexo']].count().unstack(periodo).rename(columns={'sexo':'internamento'}).sort_index(axis=1).fillna(0).astype(int)
sivep_mun_periodo.columns.names = ['qtde','ano','mes','se','dia']
sivep_mun_periodo =  sivep_mun_periodo.sort_index(level=0, axis=0, key=macro_sort)

In [ ]:
sivep_mun_periodo.to_excel('aaaa.xlsx')